# 4b. AdaBoost
We fit an Adaboost classifer with the default decision tree to see if we could get a boost in our predictive ability. Adaboost works by fitting the data and then fitting copies of that data where the weights of the incorrectly classified instances are adjusted to narrow the models focus onto the more diffcult cases. We did achieve very good results but in the end did not beat our Logistic Regression model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import pickle
import matplotlib.pyplot as plt


# Loading Time Series split assets

In [2]:
X_train = pd.read_pickle('../assets/X_train.pkl')
X_test = pd.read_pickle('../assets/X_test.pkl')
y_train = pd.read_pickle('../assets/y_train.pkl')
y_test = pd.read_pickle('../assets/y_test.pkl')

# Setting up a pipeline

In [3]:
pipe = Pipeline([
    ('ada',AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=100,min_samples_split=4))),
    
])


# Setting a range of hyperparameters


In [13]:
param_grid =  {
#  'ada__n_estimators':[],
#  'ada__learning_rate':np.linspace(.001,.95,50)
 'ada__learning_rate':[0.5,0.17,.25,.50] 
}

# Running a gridsearch

In [14]:
gs = GridSearchCV(pipe, param_grid=param_grid,verbose=1,scoring='roc_auc', cv = TimeSeriesSplit())

In [15]:
gs.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  2.5min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('ada', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
          ...0,
            warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'ada__learning_rate': [0.5, 0.17, 0.25, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

# Scoring

In [16]:
gs.score(X_train,y_train)

0.9999799869699779

In [17]:
gs.score(X_test,y_test)

0.7000205307825691

In [ ]:
X_train_preds = gs.predict(X_train)

In [ ]:
preds = pd.DataFrame({
    "preds":X_train_preds,
    "truth":y_train
})

In [ ]:
preds.sum()

In [ ]:
gs.best_params_

In [ ]:
y_train.shape

In [ ]:
X_train.shape

In [ ]:
X_train_preds.shape

In [ ]:
with open('../assets/ada_boost_model_925_322.pkl','wb+') as f:
    pickle.dump(gs,f)